Author: Elli Heyes (elli.heyes@city.ac.uk)
Last edited: 08/05/2024

In this notebook we construct a reinforcement learning algorithm to generate FRST triangulations of reflexive polytopes together with rank-5 line bundle sums that satisfy anomaly cancellation and poly-stability.
We use a deep Q-learning algorithm, where the triangulation states are encoded by 2-face triangulations. 

In [1]:
import sys
import subprocess
try:
    import keras 
    import tensorflow as tf
except:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'tensorflow'])
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'keras'])
    import keras
    import tensorflow as tf
import math
import random
import numpy as np
import matplotlib.pyplot as plt
from math import comb
from copy import deepcopy
from flint import fmpz_mat
from collections import deque
from cytools import Polytope
from cytools import fetch_polytopes
from cytools.cone import Cone
from cytools.utils import gcd_list
from cytools.polytope import poly_v_to_h
from cytools.triangulation import Triangulation
from scipy.spatial import ConvexHull
from itertools import combinations

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 784.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 16.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 16.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 15.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

2024-05-08 08:15:46.148367: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 08:15:46.157200: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-08 08:15:46.253389: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 08:15:47.517103: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_two_face_triangs(poly):
    """ Input: polyotpe
        Output: 2d list of all inequivalent 2-face triangulations """
    two_face_triangs = []
    for face in p.faces(2):
        Ts = face.as_polytope().all_triangulations(only_regular=True, only_star=False, only_fine=True, include_points_interior_to_facets=True, as_list=True)
        Ts_simps = []
        for T in Ts:
            new_simp = [[p.points_to_labels(face.points()[i]) for i in simp] for simp in T.simps()]
            Ts_simps.append(new_simp)
        two_face_triangs.append(Ts_simps)
    
    return two_face_triangs

In [3]:
def random_state(two_face_triangs, max_num_triangs, h11):
    """ Input: - 2d list of all 2-face triangulations
               - maximum number of 2-face triangulations 
               - h11 value of the CY
        Output: 3d bitlist state encoding the triangulation and line bundle sum   """
    
    # determine the dimensions of the state bitlist
    dim1 = max(len(two_face_triangs),5)
    dim2 = max(max_num_triangs,h11)
    
    # randomly generate triangulation bitlist
    T = [[0 for j in range(dim2)] for i in range(dim1)]
    for i in range(len(two_face_triangs)):
        j = random.choice(range(len(two_face_triangs[i])))
        T[i][j] = 1
    
    # randomly generate line bundle sum
    V = [[0 for j in range(dim2)] for i in range(dim1)]
    sums = [0 for i in range(h11)]
    for i in range(4):
        for j in range(h11):
            V[i][j] = random.randint(-5,5)
            sums[j] += V[i][j]
            
    # ensure that the structure group of V is S(U(1))^5 and not smaller
    embedding = False
    while not embedding:
        k5 = [random.randint(-5,5) for i in range(h11)]
        new_sums = [sums[i]+k5[i] for i in range(h11)]
        if new_sums != [0 for i in range(h11)]:
            embedding = True
            for i in range(h11):
                V[4][i] = k5[i]
            
    # combine triangulation and line bundle sum
    state = [T,V]

    return state

In [4]:
def bits_to_simps(triang, two_face_triangs):
    """ Input: - bitlist describing triangulation of 2-faces 
               - list of all 2-face triangulations
        Output: the list of two simplices                    """
    two_simps = []
    for i in range(len(triang)):
        for j in range(len(triang[i])):
            if triang[i][j] == 1:
                two_simps.append(two_face_triangs[i][j])
    
    return two_simps

In [5]:
def get_actions(two_face_triangs, h11):
    """ Input: - 2d list of all 2-face triangulations
               - h11 value of the CY
        Output: the list of all possible actions on the 
                triangulation and line bundle sum       """
    # [i,j]: choose the j-th triangulation of the i-th 2-face
    T_actions = []
    for i in range(len(two_face_triangs)):
        for j in range(len(two_face_triangs[i])):
            if len(two_face_triangs[i]) > 1:
                T_actions.append([0,i,j,0])
    
    # [i,j,k]: change the k^i_j value to k
    V_actions = []
    for i in range(5):
        for j in range(h11):
            for k in range(-5,6):
                V_actions.append([1,i,j,k])
    
    # combine action lists
    action_list = T_actions + V_actions
    
    return action_list

In [6]:
def act(state, action):
    """ Input: state and action pair
        Output: a new state obtained by acting on the input state with the action """
    new_state = deepcopy(state)
    
    if action[0] == 0:
        new_state[0][action[1]] = [0 for i in range(len(state[0][action[1]]))]
        new_state[0][action[1]][action[2]] = 1
        
    else:
        new_state[1][action[1]][action[2]] = action[3]
    
    return new_state

In [7]:
def secondary_cone(poly, two_face, simps):
    """ Input: points, simplices and dimension
        Output: secondary cone of polytope """
    face = Polytope(two_face.points())
    
    two_pts_ext = [list(pt)+[1,] for pt in face.points(optimal=True)]
    pts_ext = [list(pt)+[1,] for pt in poly.points(optimal=True)]
    
    two_simps = [[face.points_to_labels(poly.points()[simp[i]]) for i in range(3)] for simp in simps]
    two_simps = [set(s) for s in two_simps]
    
    simps = [set(s) for s in simps]
    
    m = np.zeros((2+1, 2+2), dtype=int)
    null_vecs = set()
    for i in range(len(simps)):
        for j in range(len(simps[i+1:])):
            # define the simplices
            two_s1 = two_simps[i]
            two_s2 = two_simps[j]
            s1 = simps[i]
            s2 = simps[j]

            # eunsre that the simplices have a large enough intersection
            two_comm_pts = two_s1 & two_s2
            if len(two_comm_pts) != 2:
                continue
 
            two_diff_pts = list(two_s1 ^ two_s2)
            two_comm_pts = list(two_comm_pts)
            for k,pt in enumerate(two_diff_pts):    m[:,k] = two_pts_ext[pt]
            for k,pt in enumerate(two_comm_pts):    m[:,k+2] = two_pts_ext[pt]
        
            # calculate nullspace/hyperplane inequality
            v = fmpz_mat(m.tolist()).nullspace()[0]
            v = np.array(v.transpose().tolist()[0], dtype=int)

            # ensure the sign is correct
            if v[0] < 0:
                v *= -1

            # reduce the vector
            g = gcd_list(v)
            if g != 1:
                v //= g

            # construct the full vector (including all points)
            full_v = np.zeros(len(pts_ext), dtype=int)

            diff_pts = list(s1 ^ s2)
            comm_pts = list(s1 & s2)
            for k,pt in enumerate(diff_pts):    full_v[pt] = v[k]
            for k,pt in enumerate(comm_pts):    full_v[pt] = v[k+2]

            full_v = tuple(full_v)
            if full_v not in null_vecs:
                null_vecs.add(full_v)
    
    if null_vecs == set():
        sec_cone = None
    else:         
        sec_cone = Cone(hyperplanes=list(null_vecs),check=False)
    
    return sec_cone

In [8]:
def intersection(lst1, lst2):
    """ Input: two lists
        Output: the intersection of the two lists """
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

def complement(lst1, lst2):
    """ Input: two lists
        Output: the complement of the two lists """
    lst3 = []
    for item in lst1:
        if not item in lst2:
            lst3.append(item)
    for item in lst2:
        if not item in lst1:
            lst3.append(item)
   
    return lst3

In [9]:
def combine_triangulation(poly, two_face_triangs, bitlist):
    """ Input: a polytope, the list of 2-face triangulations and the bitlist state
        Output: the combined triangulation                                         """
    
    # get the list of facets
    facets = []
    for i in range(len(poly.facets())):
        facets.append(poly.points_to_labels(poly.facets()[i].points()))
        
    # get the list of 2-face simplices
    two_face_simps = [item for sublist in bits_to_simps(bitlist, two_face_triangs) for item in sublist]
    
    # combine the simplices from the 2-face triangulations 
    simps = []
    for i in range(len(two_face_simps)):
        for j in range(i,len(two_face_simps)):
            if i!=j:
                inters = intersection(two_face_simps[i], two_face_simps[j])
                comp = complement(two_face_simps[i], two_face_simps[j])
                if len(inters) == 2:
                    comp = complement(two_face_simps[i], two_face_simps[j])
                    for k in range(len((two_face_simps))):
                        if np.array([item in two_face_simps[k] for item in comp]).all():
                            simp = sorted(inters + comp)
                            for facet in facets:
                                if np.array([item in facet for item in simp]).all():
                                    if not simp in simps:
                                        simps.append(simp)
    
    simps = [[0]+simp for simp in simps]
 
    return Triangulation(poly=poly, pts=poly._labels_not_facet, simplices=simps, check_input_simplices=False) 

In [10]:
def R(state, two_face_triangs, h11, poly): 
    """ Input: - state describing triangulation and line bundle sum
               - list of two face simplices
               - h11 value of the CY
               - reflexive polytope
        Output: the reward value between 0 and 1 for the state """
    
    # covert triangulation bitlist to list of two-face simplices
    two_face_simps = bits_to_simps(state[0], two_face_triangs)
        
    # compute the secondary cones of all the non-simplicial two-face triangulations
    sec_cones = []
    for i in range(len(two_face_simps)):
        if len(two_face_simps[i]) > 1:
            sec_cones.append(secondary_cone(p, p.faces(2)[i], two_face_simps[i]))
    sec_cones = list(set(sec_cones))
    
    # check if cone is trivial
    if sec_cones != [None]:
        # check the regularity condition
        inters = None
        for i in range(len(sec_cones)):
            if sec_cones[i] != None:
                if inters == None:
                    inters = sec_cones[i]
                else:
                    inters = inters.intersection(sec_cones[i])
        if inters.is_solid():
            FRST = 1
        else:
            FRST = 0
    else:
        FRST = 1
    
    # if FRST check E8 embedding condition 
    if FRST:
        embeddings = [0 for i in range(h11)]
        for i in range(h11):
            if state[1][0][i]+state[1][1][i]+state[1][2][i]+state[1][3][i]+state[1][4][i] == 0:
                embeddings[i] = 1
            
        # if embedding condition satisfied check anomaly cancellation condition
        if sum(embeddings) == 0:
            # combine the two-face simplices into a triangulation of the full polytope
            t = combine_triangulation(poly, two_face_triangs, state[0])
        
            # define the CY
            cy = t.get_cy()
        
            # compute the second Chern class and triple intersection numbers
            c2 = cy.second_chern_class(in_basis=True)
            dijk = cy.intersection_numbers(in_basis=True, format="dense")
         
            anomalies = [0 for i in range(h11)]
            for i in range(h11):
                c2iV = 0
                for a in range(5):
                    for j in range(h11):
                        for k in range(h11):
                            c2iV += -0.5*dijk[i][j][k]*state[1][a][j]*state[1][a][k]
                if c2iV <= c2[i]:
                    anomalies[i] = 1
                    
            # if anomaly cancellation condition satisfied check poly-stability condition
            if sum(anomalies) == 0:
                
                # define the 5 M matrices
                Ms = [[dijk[i]*state[1][a][i] for i in range(h11)] for a in range(5)]
                
                # check the poly-stability condition 
                stabilities = [0 for a in range(5)]
                for a in range(5):
                    if not (max(np.array(Ms[a]).flatten()) > 0 and min(np.array(Ms[a]).flatten()) < 0):
                        stabilities[a] = 1
                
                if sum(stabilities) == 0:
                    score = 1
                else:
                    score = 1 - (1/3)*sum(stabilities)/5
            
            else:
                score = (2/3) - (1/3)*sum(anomalies)/h11 
                
        else:
            score = (1/3) - (1/3)*sum(embeddings)/h11
            
    else:
        score = 0
    
    return score

In [11]:
def agent(state_shape, action_shape):
    """ The agent maps X-states to Y-actions."""
    learning_rate = 0.001
    my_optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    my_loss = tf.keras.losses.MeanSquaredError()
    my_metric = tf.keras.metrics.MeanAbsoluteError() 
    
    model = keras.Sequential()
    model.add(keras.layers.Dense(100, input_shape=state_shape, activation='relu'))
    model.add(keras.layers.Dense(200, activation='relu'))
    model.add(keras.layers.Dense(action_shape, activation='linear'))
    model.compile(loss=my_loss, optimizer=my_optimizer, metrics=[my_metric])
    return model

In [12]:
def train(replay_memory, model, target_model, terminated):
    learning_rate = 0.7 
    discount_factor = 0.618

    min_replay_size = 1000
    if len(replay_memory)<min_replay_size:
        return

    batch_size = 64*2
    mini_batch = random.sample(replay_memory, batch_size)
    current_states = np.array([np.array(transition[0]).flatten() for transition in mini_batch])
    current_qs_list = model.predict(current_states, verbose=0)
    new_current_states = np.array([np.array(transition[3]).flatten() for transition in mini_batch])
    future_qs_list = target_model.predict(new_current_states, verbose=0)

    X = []
    Y = []
    for index, (state, action, reward, new_state, terminated) in enumerate(mini_batch):
        if not terminated:
            max_future_q = reward+discount_factor*np.max(future_qs_list[index])
        else:
            max_future_q = reward

        current_qs = current_qs_list[index]
        current_qs[action] = (1-learning_rate)*current_qs[action]+learning_rate*max_future_q

        X.append(np.array(state).flatten())
        Y.append(current_qs)

    model.fit(np.array(X), np.array(Y), batch_size=batch_size, verbose=0, shuffle=True)

In [13]:
# retreive the list of reflexive polyoptes from KS
h11 = 2
all_polys = fetch_polytopes(h11=h11, lattice="N", as_list=True, limit=10000)
print(len(all_polys))

36


In [14]:
# set the netowrk hyperparameters
max_epsilon = 1 # you can't explore more than 100% of the time
min_epsilon = 0.01 # at a minimum, we'll always explore 1% of the time
decay = 0.01

In [15]:
# define the reflexive polytopes 
p = all_polys[3]

# get the list of all 2-face triangulations 
two_face_triangs =get_two_face_triangs(p)
    
# get the maximum number of triangulations for a 2-face
max_num_triangs = max(len(x) for x in two_face_triangs)
print("Max # 2-Face Triangulations: ",max_num_triangs)

# get the action list 
action_list = get_actions(two_face_triangs, h11)

Max # 2-Face Triangulations:  2


In [16]:
dim1 = max(len(two_face_triangs),5)
dim2 = max(max_num_triangs,h11)
print("Dim1: ",dim1)
print("Dim2: ",dim2)

state_shape =  (2*dim1*dim2,)
action_shape = len(action_list)

# initialise the main model (updated every 4 steps)
model = agent(state_shape, action_shape)

# initialise the target model (updated every 100 steps)
target_model = agent(state_shape, action_shape)
target_model.set_weights(model.get_weights())

# initialise memory
replay_memory = deque(maxlen=50_000)

# epsilon-greedy algorithm in initialized at 1 meaning every step is random at the start
epsilon = 1

Dim1:  13
Dim2:  2


/home/cytools/cytools-venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# train the deep Q-learning network for 100 episodes
steps_to_update_target_model = 0
for episode in range(100):   
    # randomly set the initial state
    state = random_state(two_face_triangs, max_num_triangs, h11)
            
    # set the initial epoch number 
    epochs = 0
    
    # continue until terminated
    terminated = False
    while not terminated:
        
        # update steps to update target model variable
        steps_to_update_target_model += 1
        
        # randomly sample a number from 0 to 1 
        # if number is less than epsilon then explore the action space
        # else exploit the learned values
        if random.uniform(0,1)<epsilon:
            # randomly sample an action from the action space
            action = random.choice(range(len(action_list)))
        else:
            # choose action with the highest predicted q value
            encoded = np.array(state).flatten()
            encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
            predicted = model.predict(encoded_reshaped, verbose=0).flatten()
            action = np.argmax(predicted)

        # define the next state from the action
        new_state = act(state, action_list[action])
        
        # compute the fitness of the next state 
        fitness = R(new_state, two_face_triangs, h11, p)
        
        # if the new state is FRST then assign a big reward and update the terminated variable
        if fitness == 1:
            reward = 10
            terminated = True
        else:
            # if the new state is not FRST then compute the reward using the reward function
            reward = R(new_state, two_face_triangs, h11, p) - R(state, two_face_triangs, h11, p)   
        
        # update the replay memory 
        replay_memory.append([state, action, reward, new_state, terminated])
        
        # update the main model
        if steps_to_update_target_model % 4 == 0 or terminated:
            train(replay_memory, model, target_model, terminated)
        
        # update the state
        state = deepcopy(new_state)
        
        if terminated:
            if steps_to_update_target_model >= 100:
                # update the target model
                target_model.set_weights(model.get_weights())
                steps_to_update_target_model = 0
                
        # update epoch number
        epochs += 1
    
    # update epsilon parameter
    epsilon = min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*episode)
    
    print("Total Num Epochs: ",epochs)        

Total Num Epochs:  101
Total Num Epochs:  508
Total Num Epochs:  118
Total Num Epochs:  30
Total Num Epochs:  117
Total Num Epochs:  240


In [232]:
# run the trained network to find FRST and line bundle sum pairs until no new pairs are found in 10 runs
finished = False
terminal_states, count = [], []
while not finished:
    # randomly set the initial state
    state = random_state(two_face_triangs, max_num_triangs, h11)
    
    # set the initial epoch number 
    epochs = 0
    
    # define the path list of states visited
    path = []

    # continue until terminated
    terminated = False
    while not terminated:
        # append the path with the current state
        path.append(state)
        
        # choose the next state as the one with the highest q value that hasn't been visited before
        new = False
        encoded = np.array(state).flatten()
        encoded_reshaped = encoded.reshape([1, encoded.shape[0]])
        q_list = target_model.predict(encoded_reshaped, verbose=0).flatten()

        while not new:
            # update epoch number
            epochs += 1
            
            # choose action with the highest predicted q value
            action = np.argmax(q_list)

            # define the next state from the action
            new_state = act(state, action_list[action])
            
            # check if the next state has been visited before
            if new_state not in path:
                new = True
            
            # update the q value of the next state in the q list
            q_list[np.argmax(q_list)] = -100
            
        # check if the next state is FRST with anomaly cancellation satisfied and if so update the terminated variable
        reward = R(new_state, two_face_triangs, h11, p)
        if reward == 1:
            print("Total Num Epochs: ",epochs)
            terminated = True
            count.append(len(terminal_states))
            if not new_state in terminal_states:
                terminal_states.append(new_state)
            else:
                "REPEAT"
                
        # update the state
        state = deepcopy(new_state)
        
    # if number of unique terminal states has not increased in the last 100 runs then finish
    if len(count) > 100:
        finished = True
    else:
        if len(count) > 10:
            if len(list(set(count[-10:]))) == 1:
                finished = True

Total Num Epochs:  49
Total Num Epochs:  8
Total Num Epochs:  6
Total Num Epochs:  16
Total Num Epochs:  1
Total Num Epochs:  5
Total Num Epochs:  8
Total Num Epochs:  3
Total Num Epochs:  5
Total Num Epochs:  6
Total Num Epochs:  9
Total Num Epochs:  12
Total Num Epochs:  3
Total Num Epochs:  24
Total Num Epochs:  3
Total Num Epochs:  40
Total Num Epochs:  9
Total Num Epochs:  12
Total Num Epochs:  10
Total Num Epochs:  95
Total Num Epochs:  18
Total Num Epochs:  12
Total Num Epochs:  2
Total Num Epochs:  72
Total Num Epochs:  11
Total Num Epochs:  5
Total Num Epochs:  3
Total Num Epochs:  19
Total Num Epochs:  6
Total Num Epochs:  30
Total Num Epochs:  8
Total Num Epochs:  1
Total Num Epochs:  14
Total Num Epochs:  7
Total Num Epochs:  2
Total Num Epochs:  14
Total Num Epochs:  5
Total Num Epochs:  77
Total Num Epochs:  5
Total Num Epochs:  16
Total Num Epochs:  2
Total Num Epochs:  3
Total Num Epochs:  16
Total Num Epochs:  59
Total Num Epochs:  3
Total Num Epochs:  10
Total Num Epo

In [237]:
def state_to_CY_V(state, poly, two_face_triangs, h11):
    T = combine_triangulation(poly, two_face_triangs, state[0])
    CY = T.get_cy()
    V = [[state[1][a][i] for i in range(h11)] for a in range(5)]
    return CY, V

In [243]:
CYsVs = [state_to_CY_V(state, p, two_face_triangs, h11) for state in terminal_states]

In [244]:
CYsVs[0]

(A Calabi-Yau 3-fold hypersurface with h11=2 and h21=74 in a 4-dimensional toric variety,
 [[4, -5], [5, -5], [4, -3], [-5, 5], [-5, 5]])